# Save a Spark MLlib model in PMML format

This notebook demonstrates saving a trained Spark MLlib model in PMML format.

This notebook runs on Spark Scala 2.11.


## Notebook sections

1. [Load and prepare training data](#loadata)
2. [Train and evaluate model](#trainmodel)
3. [Save model in PMML format](#savemodel)



**About the sample model**

The sample model built here is a logistic regression model for predicting whether or not a customer will purchase a tent from a fictional outdoor equipment store, based on the customer charateristics.

The data used to train the model is the "GoSales.csv" training data in the IBM Watson Studio community: <a href="https://dataplatform.cloud.ibm.com/exchange/public/entry/view/aa07a773f71cf1172a349f33e2028e4e" target="_blank" rel="noopener noreferrer">GoSales sample data</a>.

### <a id="loaddata"></a> 1. Load and prepare sample training data

In [2]:
// Download sample training data to notebook working directory
import scala.io.Source
import java.io.FileWriter
val url = "https://dataplatform.cloud.ibm.com/data/exchange-api/v1/entries/aa07a773f71cf1172a349f33e2028e4e/data?accessKey=e98b7315f84e5448aa94c633ca66ea83"
val filename = "GoSales.csv"
val data = Source.fromURL( url )
val data_file = new FileWriter( filename )
data_file.write( data.mkString )
data_file.close

url = https://dataplatform.cloud.ibm.com/data/exchange-api/v1/entries/aa07a773f71cf1172a349f33e2028e4e/data?accessKey=e98b7315f84e5448aa94c633ca66ea83
filename = GoSales.csv
data = empty iterator
data_file = java.io.FileWriter@18ce9481


java.io.FileWriter@18ce9481

In [3]:
// Read sample data into a Spark DataFrame
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()
val df = spark.read
    .format( "org.apache.spark.sql.execution.datasources.csv.CSVFileFormat" )
    .option( "header", "true" )
    .option( "inferSchema", "true" )
    .load( filename )
df.printSchema()

root
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)
 |-- IS_TENT: boolean (nullable = true)
 |-- PRODUCT_LINE: string (nullable = true)
 |-- PURCHASE_AMOUNT: double (nullable = true)



spark = org.apache.spark.sql.SparkSession@6c1280e5
df = [GENDER: string, AGE: int ... 5 more fields]


[GENDER: string, AGE: int ... 5 more fields]

In [4]:
// Select columns of interest
// Convert the boolean label colum, IS_TENT, to an integer (0 or 1)
val training_data  = df.selectExpr( "GENDER", "AGE", "MARITAL_STATUS", "PROFESSION", "cast( IS_TENT as integer) IS_TENT" )
training_data.printSchema()

root
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)
 |-- IS_TENT: integer (nullable = true)



training_data = [GENDER: string, AGE: int ... 3 more fields]


[GENDER: string, AGE: int ... 3 more fields]

In [5]:
// Create indexers for string columns
import org.apache.spark.ml.feature.StringIndexer
val indexer_GENDER         = new StringIndexer().setInputCol( "GENDER" ).setOutputCol( "GENDER_index" ).fit( training_data )
val indexer_MARITAL_STATUS = new StringIndexer().setInputCol( "MARITAL_STATUS" ).setOutputCol( "MARITAL_STATUS_index" ).fit( training_data )
val indexer_PROFESSION     = new StringIndexer().setInputCol( "PROFESSION" ).setOutputCol( "PROFESSION_index" ).fit( training_data )

indexer_GENDER = strIdx_30eeba0e60a9
indexer_MARITAL_STATUS = strIdx_3bc6e6eff00d
indexer_PROFESSION = strIdx_4168a53310ad


strIdx_4168a53310ad

In [6]:
// Add columns for the indexes strings
val training_data_2 = indexer_PROFESSION.transform( indexer_MARITAL_STATUS.transform( indexer_GENDER.transform( training_data ) ) )
training_data_2.show( 5 )

+------+---+--------------+------------+-------+------------+--------------------+----------------+
|GENDER|AGE|MARITAL_STATUS|  PROFESSION|IS_TENT|GENDER_index|MARITAL_STATUS_index|PROFESSION_index|
+------+---+--------------+------------+-------+------------+--------------------+----------------+
|     M| 27|        Single|Professional|      1|         0.0|                 1.0|             1.0|
|     F| 39|       Married|       Other|      0|         1.0|                 0.0|             0.0|
|     F| 39|       Married|       Other|      0|         1.0|                 0.0|             0.0|
|     F| 56|   Unspecified| Hospitality|      0|         1.0|                 2.0|             5.0|
|     M| 45|       Married|     Retired|      0|         0.0|                 0.0|             8.0|
+------+---+--------------+------------+-------+------------+--------------------+----------------+
only showing top 5 rows



training_data_2 = [GENDER: string, AGE: int ... 6 more fields]


[GENDER: string, AGE: int ... 6 more fields]

In [7]:
// For interest, view the mappings of string inputs to indexes
import org.apache.spark.sql.functions._
training_data_2.select( "GENDER", "GENDER_index" ).distinct().sort( asc( "GENDER_index" ) ).show
training_data_2.select( "MARITAL_STATUS", "MARITAL_STATUS_index" ).distinct().sort( asc( "MARITAL_STATUS_index" ) ).show
training_data_2.select( "PROFESSION", "PROFESSION_index" ).distinct().sort(  asc( "PROFESSION_index" ) ).show

+------+------------+
|GENDER|GENDER_index|
+------+------------+
|     M|         0.0|
|     F|         1.0|
+------+------------+

+--------------+--------------------+
|MARITAL_STATUS|MARITAL_STATUS_index|
+--------------+--------------------+
|       Married|                 0.0|
|        Single|                 1.0|
|   Unspecified|                 2.0|
+--------------+--------------------+

+------------+----------------+
|  PROFESSION|PROFESSION_index|
+------------+----------------+
|       Other|             0.0|
|Professional|             1.0|
|       Sales|             2.0|
|   Executive|             3.0|
|      Trades|             4.0|
| Hospitality|             5.0|
|     Student|             6.0|
|      Retail|             7.0|
|     Retired|             8.0|
+------------+----------------+



In [8]:
// Create an assembler that generates the feature vector column
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
val feature_vector_assembler = new VectorAssembler()
  .setInputCols( Array( "GENDER_index", "AGE", "MARITAL_STATUS_index", "PROFESSION_index" ) )
  .setOutputCol( "features_vector" )

feature_vector_assembler = vecAssembler_8e1c0797b26d


vecAssembler_8e1c0797b26d

In [9]:
// Update the training data to add the feature vector column
val training_data_3 = feature_vector_assembler.transform( training_data_2 )
training_data_3.show( 5 )

+------+---+--------------+------------+-------+------------+--------------------+----------------+------------------+
|GENDER|AGE|MARITAL_STATUS|  PROFESSION|IS_TENT|GENDER_index|MARITAL_STATUS_index|PROFESSION_index|   features_vector|
+------+---+--------------+------------+-------+------------+--------------------+----------------+------------------+
|     M| 27|        Single|Professional|      1|         0.0|                 1.0|             1.0|[0.0,27.0,1.0,1.0]|
|     F| 39|       Married|       Other|      0|         1.0|                 0.0|             0.0|[1.0,39.0,0.0,0.0]|
|     F| 39|       Married|       Other|      0|         1.0|                 0.0|             0.0|[1.0,39.0,0.0,0.0]|
|     F| 56|   Unspecified| Hospitality|      0|         1.0|                 2.0|             5.0|[1.0,56.0,2.0,5.0]|
|     M| 45|       Married|     Retired|      0|         0.0|                 0.0|             8.0|[0.0,45.0,0.0,8.0]|
+------+---+--------------+------------+-------+

training_data_3 = [GENDER: string, AGE: int ... 7 more fields]


[GENDER: string, AGE: int ... 7 more fields]

### <a id="trainmodel"></a> 2. Create a logistic regression model and then train and evaluate the model

In [10]:
// Create a logistic regression model
import org.apache.spark.ml.classification.LogisticRegression
val lr = new LogisticRegression()
  .setFeaturesCol( "features_vector" )
  .setLabelCol( "IS_TENT" )

lr = logreg_eeae090a6c13


logreg_eeae090a6c13

In [11]:
// Split the training data into a training set and a test set
val splits = training_data_3.randomSplit( Array( 0.75, 0.25 ), seed = 2019 )
val train = splits( 0 ).cache()
val test = splits( 1 )
println( "Train count: " + train.count() )
println( "Test count: "  + test.count()  )

Train count: 45186
Test count: 15066


splits = Array([GENDER: string, AGE: int ... 7 more fields], [GENDER: string, AGE: int ... 7 more fields])
train = [GENDER: string, AGE: int ... 7 more fields]
test = [GENDER: string, AGE: int ... 7 more fields]


[GENDER: string, AGE: int ... 7 more fields]

In [12]:
// Train the model
val lr_model = lr.fit( train )
println( s"Coefficients: ${lr_model.coefficients} Intercept: ${lr_model.intercept}" )

Coefficients: [-1.6691109760782625,-0.13800412307651586,-0.24480344066228285,0.22211090608184475] Intercept: 3.5810705512843337


lr_model = logreg_eeae090a6c13


logreg_eeae090a6c13

In [13]:
// Evaluate the model performance
val predictions = lr_model.transform( test )
val correct_false = predictions
  .filter( "IS_TENT == 0 AND prediction == 0.0" )
  .select( "GENDER", "AGE", "MARITAL_STATUS", "PROFESSION", "IS_TENT", "prediction" )
val correct_true = predictions
  .filter( "IS_TENT == 1 AND prediction != 0.0" )
  .select( "GENDER", "AGE", "MARITAL_STATUS", "PROFESSION", "IS_TENT", "prediction" )
println( "Success rate: " + Math.round( 100 * ( correct_false.count() + correct_true.count() ).toFloat / predictions.count() ) + "%" )

Success rate: 78%


predictions = [GENDER: string, AGE: int ... 10 more fields]
correct_false = [GENDER: string, AGE: int ... 4 more fields]
correct_true = [GENDER: string, AGE: int ... 4 more fields]


[GENDER: string, AGE: int ... 4 more fields]

In [53]:
// Grab some example data for quick test
println( "Customer who did not buy a tent:\n" + training_data_3.rdd.take(14).last )
println( "\nCustomer who did buy a tent:\n " + training_data_3.rdd.take(15).last )

Customer who did not buy a tent:
[F,35,Married,Professional,0,1.0,0.0,1.0,[1.0,35.0,0.0,1.0]]

Customer who did buy a tent:
 [M,20,Single,Sales,1,0.0,1.0,2.0,[0.0,20.0,1.0,2.0]]


In [55]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.linalg.Vectors
import spark.implicits._
case class Customer( features_vector: org.apache.spark.ml.linalg.Vector )
val negative_example_payload = Seq( Customer( Vectors.dense(1.0, 35.0, 0.0, 1.0) ) ).toDS()
val positive_example_payload = Seq( Customer( Vectors.dense(0.0, 20.0, 1.0, 2.0) ) ).toDS()

defined class Customer
negative_example_payload = [features_vector: vector]
positive_example_payload = [features_vector: vector]


[features_vector: vector]

In [47]:
lr_model.transform( negative_example_payload ).select( "prediction" ).show()

+----------+
|prediction|
+----------+
|       0.0|
+----------+



In [48]:
lr_model.transform( positive_example_payload ).select( "prediction" ).show()

+----------+
|prediction|
+----------+
|       1.0|
+----------+



### <a id="savemodel"></a> 3. Save the model in PMML format

In [15]:
// Create an org.apache.spark.mllib.classification.LogisticRegressionModel object
import org.apache.spark.mllib.classification.LogisticRegressionModel
import org.apache.spark.mllib.linalg.Vectors
val lr_mllib = new LogisticRegressionModel( org.apache.spark.mllib.linalg.Vectors.dense( lr_model.coefficients.toArray ), lr_model.intercept )

lr_mllib = org.apache.spark.mllib.classification.LogisticRegressionModel: intercept = 3.5810705512843337, numFeatures = 4, numClasses = 2, threshold = 0.5


org.apache.spark.mllib.classification.LogisticRegressionModel: intercept = 3.5810705512843337, numFeatures = 4, numClasses = 2, threshold = 0.5

In [16]:
// Save the model to a file in PMML format
lr_mllib.toPMML( "spark-mllib-lr-model-pmml.xml" )

In [18]:
import sys.process._
"more spark-mllib-lr-model-pmml.xml" !

::::::::::::::
spark-mllib-lr-model-pmml.xml
::::::::::::::
<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<PMML xmlns="http://www.dmg.org/PMML-4_2" version="4.2">
    <Header description="logistic regression">
        <Application name="Apache Spark MLlib" version="2.3.0"/>
        <Timestamp>2019-01-12T21:42:18</Timestamp>
    </Header>
    <DataDictionary numberOfFields="5">
        <DataField name="field_0" optype="continuous" dataType="double"/>
        <DataField name="field_1" optype="continuous" dataType="double"/>
        <DataField name="field_2" optype="continuous" dataType="double"/>
        <DataField name="field_3" optype="continuous" dataType="double"/>
        <DataField name="target" optype="categorical" dataType="string"/>
    </DataDictionary>
    <RegressionModel modelName="logistic regression" functionName="classification" normalizationMethod="logit">
        <MiningSchema>
            <MiningField name="field_0" usageType="active"/>
            <MiningFie

0

**Tip**

You can use your mouse to highlight-copy the PMML content from running the previous cell, then paste the content into a text editor on your local computer, and then save the file on your local computer as "spark-mllib-lr-model-pmml.xml"

## Summary and next steps
In this notebook, you created a logistic regression model using Spark MLlib and then saved the model to a file in PMML format.

To learn how you can import this model into Watson Machine Learning, see:
<a href="https://dataplatform.cloud.ibm.com/docs/content/analyze-data/ml-import-pmml.html" target="_blank" rel="noopener noreferrer">Importing models into Watson Machine Learning from PMML</a>

### <a id="authors"></a>Authors

**Sarah Packowski** is a member of the IBM Watson Studio Content Design team in Canada.


<hr>
Copyright &copy; IBM Corp. 2019. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>